In [ ]:
from shutil import which
print(which('espeak'))
print(which('espeak-ng'))

None
/usr/bin/espeak-ng


In [ ]:
import os
import gdown
import pandas as pd
import soundfile as sf
import shutil
import bangla


try:
  from TTS.utils.synthesizer import Synthesizer
except:
  print("coundn't import TTS synthesizer,trying again!")

import TTS

import time
from typing import List

import numpy as np
import pysbd
import torch

from TTS.config import load_config
from TTS.tts.models import setup_model as setup_tts_model

from TTS.tts.utils.synthesis import synthesis, transfer_voice, trim_silence
from TTS.utils.audio import AudioProcessor
from TTS.vocoder.models import setup_model as setup_vocoder_model
from TTS.vocoder.utils.generic_utils import interpolate_vocoder_input

In [ ]:
!nvidia-smi
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import torch
# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.models.vits import Vits, VitsAudioConfig
from TTS.tts.utils.text.phonemizers import ESpeak
# BaseDatasetConfig: defines name, formatter and path of the dataset.


from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

from TTS.tts.configs.shared_configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig
#from TTS.configs import BaseDatasetConfig,BaseAudioConfig,CharactersConfig#GlowTTSConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.speakers import SpeakerManager

from TTS.config import load_config

from trainer import Trainer, TrainerArgs

Sun May 19 06:39:40 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A5000               On  | 00000000:00:0A.0 Off |                  Off |
| 30%   34C    P2              58W / 230W |   4048MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
pretrained = True

pretrained_path = ''
if(pretrained):
    pretrained_path = 'vits_20k_steps'

meta_file = 'content/mono/metadata_udoy_lj.txt'
root_path = 'content/mono'

def formatter(root_path, meta_file, **kwargs):  # pylint: disable=unused-argument
    """Normalizes the LJSpeech meta data file to TTS format
    https://keithito.com/LJ-Speech-Dataset/"""
    txt_file = meta_file
    items = []
    speaker_name = "ljspeech"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = os.path.join(root_path, "wav", cols[0] + ".wav")
            try:
                text = cols[1]
            except:
                print("not found")

            items.append({"text": text, "audio_file": wav_file, "speaker_name": speaker_name, "root_path": root_path})
    return items


dataset_config = BaseDatasetConfig(
     meta_file_train=meta_file, path=os.path.join(root_path, "")
)

In [ ]:
import pprint
pprint.pp(dataset_config)

BaseDatasetConfig(formatter='',
                  dataset_name='',
                  path='content/mono/',
                  meta_file_train='content/mono/metadata_udoy_lj.txt',
                  ignored_speakers=None,
                  language='',
                  phonemizer='',
                  meta_file_val='',
                  meta_file_attn_mask='')


In [ ]:
characters_config = CharactersConfig(
  pad = '<PAD>',
  eos = '।', #'<EOS>', #'।',
  bos = '<BOS>',# None,
  blank = '<BLNK>',
  characters =  "তট৫ভিঐঋখঊড়ইজমএেঘঙসীঢ়হঞ‘ঈকণ৬ঁৗশঢঠ\u200c১্২৮দৃঔগও—ছউংবৈঝাযফ\u200dচরষঅৌৎথড়৪ধ০ুূ৩আঃপয়’নলো",
  phonemes =  "ˌamɾʃˈonbŋlitjʰɔdkpeɟːfɡuhrʈæsʒɖwəcã",
  punctuations = "-!,|.? ",
)

In [ ]:
output_path = ''

audio_config = VitsAudioConfig(
    sample_rate=22050,
    win_length=1024,
    hop_length=256,
    num_mels=80,
    mel_fmin=0,
    mel_fmax=None
)

config = VitsConfig(
    audio=audio_config,
    run_name="VITS",
    batch_size=8,
    eval_batch_size=8,
    batch_group_size=0,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1,
    phonemizer='espeak',
    text_cleaner='phoneme_cleaners',
    use_phonemes=True,
    phoneme_language="bn",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    compute_input_seq_cache=True,
    add_blank=True,
    use_language_weighted_sampler = True,
    print_step=50,
    print_eval=False,
    mixed_precision=False,
    output_path=output_path,
    datasets=[dataset_config],
    characters = characters_config,
    save_step=1000,
    cudnn_benchmark=True,
    test_sentences = [
        'আমার   সোনার বাংলা, আমি তোমায় ভালোবাসি।',
        'চিরদিন   তোমার আকাশ, তোমার বাতাস, আমার প্রাণে বাজায় বাঁশি',
        'ও মা,   ফাগুনে তোর আমের বনে ঘ্রাণে পাগল করে,মরি হায়, হায় রে।'
    ]
)

In [ ]:
ap = AudioProcessor.init_from_config(config)

tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [ ]:
# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config,formatter=formatter, eval_split=True)

 | > Found 649 files in /workspace/content/mono


In [ ]:
print(len(train_samples),len(eval_samples))

643 6


In [ ]:
model = Vits(config, ap, tokenizer, speaker_manager=None)

In [ ]:
trainer = Trainer(
    TrainerArgs(continue_path = pretrained_path),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples
)

fatal: not a git repository (or any of the parent directories): .git
 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 64
 | > Num. of Torch Threads: 32
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: True
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=vits_20k_steps
 > Restoring from checkpoint_24229.pth ...
 > Restoring Model...
 > Restoring Optimizer...
 > Restoring Scaler...
 > Model restored from step 24229

 > Model has 83043052 parameters


In [ ]:
trainer.config.test_sentences = [
        'আমার সোনার বাংলা, আমি তোমায় ভালোবাসি।',
        'চিরদিন তোমার আকাশ, তোমার বাতাস, আমার প্রাণে বাজায় বাঁশি',
        'ও মা, ফাগুনে তোর আমের বনে ঘ্রাণে পাগল করে,মরি হায়, হায় রে।'
    ]

In [ ]:
# pprint.pp(trainer.config)

In [ ]:
%%time

trainer.fit()

 > Restoring best loss from best_model_20099.pth ...
 > Starting with loaded last best loss {'train_loss': None, 'eval_loss': 15.714713017145792}

 > EPOCH: 0/300
 --> vits_20k_steps




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: bn
		| > phoneme backend: espeak
| > Number of instances : 643



 > TRAINING (2024-05-19 06:40:00) 


 | > Preprocessing samples
 | > Max text length: 171
 | > Min text length: 5
 | > Avg text length: 61.76049766718507
 | 
 | > Max audio length: 264600
 | > Min audio length: 9794
 | > Avg audio length: 108190.52566096422
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


/opt/conda/lib/python3.10/site-packages/torch/functional.py:660: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025847130/work/aten/src/ATen/native/SpectralOps.cpp:874.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]

 > EVALUATION 



...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 3.691071033477783 (+0.21788334846496582)
     | > avg_loss_disc: 2.397871732711792 (-0.3909335136413574)
     | > avg_loss_disc_real_0: 0.23795199394226074 (-0.05883610248565674)
     | > avg_loss_disc_real_1: 0.2164091318845749 (-0.05293406546115875)
     | > avg_loss_disc_real_2: 0.20697490870952606 (-0.02441735565662384)
     | > avg_loss_disc_real_3: 0.17220616340637207 (-0.08703428506851196)
     | > avg_loss_disc_real_4: 0.1690916121006012 (-0.07965344190597534)
     | > avg_loss_disc_real_5: 0.16963349282741547 (-0.17011411488056183)
     | > avg_loss_0: 2.397871732711792 (-0.3909335136413574)
     | > avg_loss_gen: 2.1429202556610107 (-0.10562682151794434)
     | > avg_loss_kl: 1.893460750579834 (-0.08814430236816406)
     | > avg_loss_feat: 5.156439781188965 (+1.3255915641784668)
     | > avg_loss_mel: 20.574007034301758 (+0.6536159515380859)
     | > avg_loss_duration: 1.4232341051101685 (-0.02486264705657959)
     | > avg_los

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 3.426903009414673 (-0.26416802406311035)
     | > avg_loss_disc: 2.5186686515808105 (+0.12079691886901855)
     | > avg_loss_disc_real_0: 0.25370195508003235 (+0.015749961137771606)
     | > avg_loss_disc_real_1: 0.27698221802711487 (+0.06057308614253998)
     | > avg_loss_disc_real_2: 0.20323722064495087 (-0.0037376880645751953)
     | > avg_loss_disc_real_3: 0.22687630355358124 (+0.05467014014720917)
     | > avg_loss_disc_real_4: 0.18080531060695648 (+0.011713698506355286)
     | > avg_loss_disc_real_5: 0.19078296422958374 (+0.021149471402168274)
     | > avg_loss_0: 2.5186686515808105 (+0.12079691886901855)
     | > avg_loss_gen: 2.226999282836914 (+0.08407902717590332)
     | > avg_loss_kl: 2.253603458404541 (+0.36014270782470703)
     | > avg_loss_feat: 4.323004722595215 (-0.83343505859375)
     | > avg_loss_mel: 20.737171173095703 (+0.1631641387939453)
     | > avg_loss_duration: 1.4572352170944214 (+0.03400111198425293)
     | >

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 3.496586322784424 (+0.06968331336975098)
     | > avg_loss_disc: 2.542799472808838 (+0.024130821228027344)
     | > avg_loss_disc_real_0: 0.1914556473493576 (-0.062246307730674744)
     | > avg_loss_disc_real_1: 0.2283925861120224 (-0.04858963191509247)
     | > avg_loss_disc_real_2: 0.17780478298664093 (-0.025432437658309937)
     | > avg_loss_disc_real_3: 0.18806928396224976 (-0.03880701959133148)
     | > avg_loss_disc_real_4: 0.20909567177295685 (+0.028290361166000366)
     | > avg_loss_disc_real_5: 0.25818517804145813 (+0.06740221381187439)
     | > avg_loss_0: 2.542799472808838 (+0.024130821228027344)
     | > avg_loss_gen: 2.1396982669830322 (-0.08730101585388184)
     | > avg_loss_kl: 2.036327600479126 (-0.21727585792541504)
     | > avg_loss_feat: 5.204678535461426 (+0.8816738128662109)
     | > avg_loss_mel: 19.83387565612793 (-0.9032955169677734)
     | > avg_loss_duration: 1.4663910865783691 (+0.009155869483947754)
     | > 

In [ ]:
# chpt = torch.load('vits_20k_steps/checkpoint_24229.pth')

In [ ]:
# chpt['model_loss'] = 15.714713017145792

In [ ]:
# torch.save(chpt, 'vits_20k_steps/checkpoint_24229.pth')

In [ ]:
# chpt2 = torch.load('vits_20k_steps/checkpoint_20000.pth')
# chpt2['model_loss']

15.714713017145792

In [ ]:
trainer.save_checkpoint()

In [ ]:
# test_ckpt = 'vits_20k_steps/checkpoint_24229.pth'
# test_config = 'vits_20k_steps/config.json'

# !tts --text "জামিনে ছাড়া পেয়েই অপরাধে জড়িত, কিশোরকে অপহরণ" \
#       --model_path $test_ckpt \
#       --config_path $test_config \
#       --out_path out.wav

 > Using model: vits
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: জামিনে ছাড়া পেয়েই অপরাধে জড়িত, কিশোরকে অপহরণ
 > Text splitted to sentences.
['জামিনে ছাড়া পেয়েই অপরাধে জড়িত, কিশোরকে অপহরণ']
 > Processing time: 3.1359007358551025
 > Real-time factor: 0.6110085114661832
 > Saving output to o

In [ ]:
# import IPython
# IPython.display.Audio("out.wav")

In [ ]:
# watch -n0.1 nvidia-smi